In [18]:
%cd "C:\Users\user\Documents\UST\BDT\MSBD5005\project\data\"

C:\Users\user\Documents\UST\BDT\MSBD5005\project\data


In [42]:
!pip install textblob
!pip install wordcloud


[notice] A new release of pip available: 22.2.2 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 153.1/153.1 kB 8.9 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [85]:
from textblob import TextBlob
import nltk
import pandas as pd
import re
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [93]:
# file = './daily/tweet_verified_221130-230418.csv'
# df = pd.read_csv(file)
# df = df[['tid', 'renderedContent']].copy(deep=True)


file = './tweet_combine_221130-230315.pkl'
with open(file, 'rb') as f:
    tweets_dict = pickle.load(f)
arr = []
for tid, tweet in tweets_dict.items():
    arr.append((tid, tweet.renderedContent))

df = pd.DataFrame(arr, columns=['tid', 'renderedContent'])

In [95]:
df.head()

,tid,renderedContent
0,1631081841687162881,@Kate_Frome @JeremyPoxon @Tony_Burke I remembe...
1,1631081831784411136,So now that Open AIs ChatGPT is available via ...
2,1631081824754581505,"Heads up y'all, I recently ran into a similar ..."
3,1631081809990631426,Can ChatGPT do my taxes? 👀
4,1631081808589643776,@andrewwhite01 I have seen reviews like this p...


In [96]:
# credit to Hayden
def remove_urls_and_entities(text):
    """
    Removes URLs and HTML entities from a string using regular expressions.
    
    Args:
        text (str): The input string to remove URLs and HTML entities from.
        
    Returns:
        str: The input string with any URLs and HTML entities removed.
    """
    # Define regular expressions to match URLs and HTML entities
    url_pattern = re.compile(r'https?://(?:www\.\S+|(?!www)\S+)')
    entity_pattern = re.compile(r'&\w+;')
    
    # Use the sub() method to replace URLs and HTML entities with an empty string
    text_without_urls_and_entities = url_pattern.sub('', text)
    text_without_urls_and_entities = entity_pattern.sub('', text_without_urls_and_entities)
    
    return text_without_urls_and_entities

In [97]:
def remove_stopword(s: str) -> str:
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(s)
    # converts the words in word_tokens to lower case and then checks whether 
    #they are present in stop_words or not
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    #with no lower case conversion
    filtered_sentence = []
    
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)

    return ' '.join(filtered_sentence)

In [98]:
def extract_noun(s: str) -> list[str]:
    s = remove_urls_and_entities(s)
    s = remove_stopword(s)
    return ' '.join([item[0] for item in list(filter(lambda tag: tag[1] in ['NN', 'NNS', 'NNP', 'NNPS'], TextBlob(s).tags))])

In [99]:
df['renderedContentNoun'] = df['renderedContent'].apply(extract_noun)

In [101]:
if file.endswith('.csv'):
    df.to_csv(re.sub('.csv$', '_noun.csv', file))
elif file.endswith('.pkl'):
    df.to_csv(re.sub('.pkl$', '_noun.csv', file))